In [2]:
#Importing libraries

import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests

print("Libraries imported successfully......")

Libraries imported successfully......


In [3]:
teams = pd.read_csv('expanded_teams_data.csv')

print("Data loaded successfully..")

Data loaded successfully..


In [4]:
teams

,code,team_id,team_name,team_short_name,unavailable,pulse_id,event_id,fixture_id,fixture_difficulty,opponent_team,opponent_score,team_score,Game_played,is_home
0,3,1,Arsenal,ARS,False,1,1,2.0,2.0,16.0,1.0,2.0,True,True
1,7,2,Aston Villa,AVL,False,2,1,6.0,4.0,15.0,5.0,1.0,True,False
2,91,3,Bournemouth,BOU,False,127,1,3.0,2.0,19.0,1.0,1.0,True,True
3,94,4,Brentford,BRE,False,130,1,8.0,3.0,18.0,2.0,2.0,True,True
4,36,5,Brighton,BHA,False,131,1,4.0,2.0,12.0,1.0,4.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,17,16,Nott'm Forest,NFO,False,15,38,374.0,2.0,6.0,NaN,NaN,False,False
756,49,17,Sheffield Utd,SHU,False,18,38,380.0,3.0,18.0,NaN,NaN,False,True
757,6,18,Spurs,TOT,False,21,38,380.0,2.0,17.0,NaN,NaN,False,False
758,21,19,West Ham,WHU,False,25,38,379.0,5.0,13.0,NaN,NaN,False,False


Games played


In [5]:
# Convert 'Game_played' column to integers (1 for True, 0 for False)
teams['Game_played'].fillna(False, inplace=True)
teams['Game_played'] = teams['Game_played'].astype(int)

# Calculate cumulative 'Games_Count' for each row
teams['Games_Count'] = teams.groupby(['team_id', 'team_name'])['Game_played'].cumsum()



Total goals

In [6]:
teams['Total_Goals'] = teams.groupby(['team_id', 'team_name'])['team_score'].cumsum()


Goals per game

In [7]:
teams['Goals_per_game'] = teams['Total_Goals'] / teams['Games_Count']

Goals conceded

In [8]:
teams['Total_Conceded'] = teams.groupby(['team_id', 'team_name'])['opponent_score'].cumsum()


In [9]:
Liverpool = teams[teams['team_id'] == 11]
Liverpool

,code,team_id,team_name,team_short_name,unavailable,pulse_id,event_id,fixture_id,fixture_difficulty,opponent_team,opponent_score,team_score,Game_played,is_home,Games_Count,Total_Goals,Goals_per_game,Total_Conceded
10,14,11,Liverpool,LIV,False,10,1,9.0,3.0,7.0,1.0,1.0,1,False,1,1.0,1.000000,1.0
30,14,11,Liverpool,LIV,False,10,2,14.0,2.0,3.0,1.0,3.0,1,True,2,4.0,2.000000,2.0
50,14,11,Liverpool,LIV,False,10,3,29.0,4.0,15.0,1.0,2.0,1,False,3,6.0,2.000000,3.0
70,14,11,Liverpool,LIV,False,10,4,37.0,3.0,2.0,0.0,3.0,1,True,4,9.0,2.250000,3.0
90,14,11,Liverpool,LIV,False,10,5,50.0,2.0,20.0,1.0,3.0,1,False,5,12.0,2.400000,4.0
110,14,11,Liverpool,LIV,False,10,6,57.0,2.0,19.0,1.0,3.0,1,True,6,15.0,2.500000,5.0
130,14,11,Liverpool,LIV,False,10,7,68.0,3.0,18.0,2.0,1.0,1,False,7,16.0,2.285714,7.0
150,14,11,Liverpool,LIV,False,10,8,72.0,3.0,5.0,NaN,NaN,0,False,7,NaN,NaN,NaN
170,14,11,Liverpool,LIV,False,10,9,85.0,2.0,9.0,NaN,NaN,0,True,7,NaN,NaN,NaN
190,14,11,Liverpool,LIV,False,10,10,97.0,2.0,16.0,NaN,NaN,0,True,7,NaN,NaN,NaN
